In [1]:
import tensorflow as tf
import numpy as np

# Size of the hidden state.
HIDDEN_SIZE = 4

# Size of the MANN memory.
MEMORY_SIZE = 6

# Batch size.
BATCH_SIZE = 2

# Length of sequence (= single recurrent layer of number of units)
SEQ_LENGTH = 3 # in fact 4 elements, but: 0->1, 1->2, 2->3

# Number of smallest elements.
N_SMALLEST =2


In [32]:
def create_feed_dict(batch_seq):
    """Creates a dictionaries for different sets: maps data onto Tensor placeholders."""
    feed_dict = dict()
    # Feed batch to input buffers.
    for i in range(SEQ_LENGTH + 1):
        feed_dict[data_buffers[i]] = batch_seq[i]
    # Reset previous state and output
    #feed_dict[prev_output] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
    #feed_dict[prev_state] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
    
    return feed_dict # {prev_output: train_output_zeros, prev_state: train_state_zeros }

In [34]:
# Reset graph - just in case.
tf.reset_default_graph()

# Memory.
memory = tf.Variable(tf.zeros([HIDDEN_SIZE, MEMORY_SIZE]), trainable=False, name="Memory_M")
# SET INITIAL MEMORY STATE.
memory_set = memory.assign(tf.transpose([
    [0.0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [1, 0, 1, 0],
    [1, 1, 0, 0]]))


# Placeholders for inputs.
with tf.name_scope("Input_data"):
    # Define input data buffers.
    data_buffers = list()
    for _ in range(SEQ_LENGTH + 1):
        # Collect placeholders for inputs/labels.
        data_buffers.append(tf.placeholder(tf.float32, shape=None, name="data_buffers"))
    print ("data_buffers shape =", data_buffers[0].shape)

    # Sequence of batches.
    input_seq_batch = data_buffers[:SEQ_LENGTH]
    print ("Seq length  =", len(input_seq_batch))
    print ("Batch shape =", input_seq_batch[0].shape)

    # Labels are pointing to the same placeholders!
    # Labels are inputs shifted by one time step.
    labels_seq_batch = data_buffers[1:]  
    # Concatenate targets into 2D tensor.
    target_batch = tf.concat(labels_seq_batch, 0)

# Create SEQ_LEN x BATCH_SIZE placeholders for similarity - each MEMORY_SIZE x 1,  
with tf.name_scope("similarity"):

    # Normalize sequence of batches.
    norm_seq_batch = list()
    for i_seq in range(SEQ_LENGTH):
        # Collect placeholders for inputs/labels.
        norm_seq_batch.append(tf.nn.l2_normalize(input_seq_batch[i_seq],1) )
    
    # Normalize memory.
    norm_memory = tf.nn.l2_normalize(memory,0)
    print("norm_memory =", norm_memory)
    
    # Calculate cosine similarity.
    similarity_seq_batch = list()    
    for i_seq in range(SEQ_LENGTH):
        similarity_seq_batch.append(tf.matmul(norm_seq_batch[i_seq], norm_memory))
    
    # Calcualte read weights based on similarity.
    read_weights_seq_batch = list()    
    for i_seq in range(SEQ_LENGTH):
        read_weights_seq_batch.append(tf.nn.softmax(similarity_seq_batch[i_seq]))    
    
# Finally - initialize all variables.
initialize_model = tf.global_variables_initializer()    

data_buffers shape = <unknown>
Seq length  = 3
Batch shape = <unknown>
norm_memory = Tensor("similarity/l2_normalize_3:0", shape=(4, 6), dtype=float32)


In [36]:

########################
# Execute graph.
sess=tf.InteractiveSession()
# Initialize.
sess.run(initialize_model)
memory_, norm_memory_ = sess.run([memory_set, norm_memory])
print("Memory =\n",memory_)
print("norm_memory_ =\n",norm_memory_)

# Batch - of dimensions: SEQUENCE x BATCH x VECTOR SIZE
batch_seq = np.array([[[0, 0, 1, 0],[0, 0, 1, 0]],
             [[0, 1, 0, 0],[1, 0, 0, 0]],
             [[0, 0, 1, 0],[0,1,0,0]],
            [[0, 0, 1, 0],[0,1,0,0]]])

#batch_seq = np.array([[[0, 0, 1, 0],[0, 1, 0, 0],[0, 0, 1, 0]],
#             [[0, 0, 1, 0],[1, 0, 0, 0],[0,1,0,0]]])

print("Batch=\n",batch_seq.shape)

input_seq_batch_, norm_seq_batch_, similarity_seq_batch_, read_weights_seq_batch_ = sess.run([
    input_seq_batch, norm_seq_batch, similarity_seq_batch, read_weights_seq_batch],
    feed_dict=create_feed_dict(batch_seq))

print("inps.len=",len(input_seq_batch_))
for i in range(SEQ_LENGTH):
    print("inputs[",i, "] =\n",input_seq_batch_[i])

for i in range(SEQ_LENGTH):
    print("norm_batch[",i, "] =\n",norm_seq_batch_[i])

for i in range(SEQ_LENGTH):
    print("similarity_seq_batch_[",i, "] = ",similarity_seq_batch_[i])

for i in range(SEQ_LENGTH):
    print("read_weights_seq_batch_[",i, "] = ",read_weights_seq_batch_[i])


Memory =
 [[ 0.  0.  0.  1.  1.  1.]
 [ 0.  0.  1.  0.  0.  1.]
 [ 0.  1.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  0.  0.]]
norm_memory_ =
 [[ 0.          0.          0.          0.99999994  0.70710677  0.70710677]
 [ 0.          0.          0.99999994  0.          0.          0.70710677]
 [ 0.          0.99999994  0.          0.          0.70710677  0.        ]
 [ 0.99999994  0.          0.          0.          0.          0.        ]]
Batch=
 (4, 2, 4)
inps.len= 3
inputs[ 0 ] =
 [[ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]]
inputs[ 1 ] =
 [[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]
inputs[ 2 ] =
 [[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]]
norm_batch[ 0 ] =
 [[ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]]
norm_batch[ 1 ] =
 [[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]]
norm_batch[ 2 ] =
 [[ 0.  0.  1.  0.]
 [ 0.  1.  0.  0.]]
similarity_seq_batch_[ 0 ] =  [[ 0.          0.99999994  0.          0.          0.70710677  0.        ]
 [ 0.          0.99999994  0.          0.          0.70710677  0.        ]]
similarity_seq_ba